In [14]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

def get_openmeteo_data(latitude, longitude, start_date, end_date, variables, timezone="GMT", model="gfs_seamless"):
    """
    Fetch and process weather data from the Open-Meteo API.

    Args:
        latitude (float): Latitude of the location.
        longitude (float): Longitude of the location.
        start_date (str): Start date in 'YYYY-MM-DD' format.
        end_date (str): End date in 'YYYY-MM-DD' format.
        variables (list): List of variables to fetch (e.g., 'temperature_2m_max', 'precipitation_sum').
        timezone (str): Timezone of the data (default: "GMT").
        model (str): Weather model to use (default: "gfs_seamless").

    Returns:
        pd.DataFrame: DataFrame containing requested variables and their values.
    """
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "daily": variables,
        "timezone": timezone,
        "start_date": start_date,
        "end_date": end_date,
        "models": model
    }
    
    # Fetch data from the Open-Meteo API
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]  # Process the first response (can be extended for multiple locations)
    
    # Extract daily data
    daily = response.Daily()
    data = {}
    
    # Assign requested variables dynamically
    for i, variable in enumerate(variables):
        data[variable] = daily.Variables(i).ValuesAsNumpy()
    
    # Add date range
    data["date"] = pd.date_range(
        start=pd.to_datetime(daily.Time(), unit="s", utc=True),
        end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
        freq=pd.Timedelta(seconds=daily.Interval()),
        inclusive="left"
    )
    
    # Convert to DataFrame
    df = pd.DataFrame(data)
    return df

In [ ]:
# Example Usage
latitude = 36.721962
longitude = -119.236272
start_date = "2024-11-01"
end_date = "2024-11-30"
# Select only the variables you need
variables = ["temperature_2m_max", "temperature_2m_min","apparent_temperature_max", "apparent_temperature_min",]  

In [21]:
weather_data = get_openmeteo_data(latitude, longitude, start_date, end_date, variables)
weather_data

,temperature_2m_max,temperature_2m_min,apparent_temperature_max,apparent_temperature_min,date
0,16.282499,6.182500,14.272223,3.501085,2024-11-01 00:00:00+00:00
1,15.982500,10.482500,13.559713,6.824161,2024-11-02 00:00:00+00:00
2,15.282500,6.482500,12.545670,3.156012,2024-11-03 00:00:00+00:00
3,17.632500,7.582500,15.233881,4.321394,2024-11-04 00:00:00+00:00
4,17.382500,8.632501,15.134789,6.130306,2024-11-05 00:00:00+00:00
5,17.532499,8.682500,14.906065,5.999491,2024-11-06 00:00:00+00:00
6,18.182499,7.882500,14.619677,4.620997,2024-11-07 00:00:00+00:00
7,19.332499,9.582500,16.086096,5.762269,2024-11-08 00:00:00+00:00
8,19.582499,9.232500,15.969000,5.764812,2024-11-09 00:00:00+00:00
9,20.332499,9.432500,17.299969,5.730744,2024-11-10 00:00:00+00:00
